In [64]:
import pandas as pd
import torchtext
import random
from torchtext.data import TabularDataset
import numpy as np
import os
from torchtext import data
import torch.nn as nn
import torch
import torch.optim as optim
import time
from sklearn.metrics import roc_auc_score,accuracy_score
import spacy
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
# hyperparams
TEXT_LENGTH = 100
EMBEDDING_SIZE = 50
BATCH_SIZE = 64
VOCAB_SIZE=20000

In [5]:
filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
nlp = spacy.load("en")
def tokenizer(text):
    filtered = ''.join([c if c not in filters else '' for c in text])
    return [token.text for token in nlp.tokenizer(filtered) if not token.is_space]

In [6]:
random.seed(1234)
TEXT = data.Field(lower=True, batch_first=True,fix_length=TEXT_LENGTH, preprocessing=None, tokenize=tokenizer)
LABEL = data.Field(sequential=False,is_target=True, use_vocab=False, pad_token=None, unk_token=None)

datafields = [('id', None),
              ('comment_text', TEXT), 
              ("toxic", LABEL), 
              ("severe_toxic", LABEL),
              ('obscene', LABEL), 
              ('threat', LABEL),
              ('insult', LABEL),
              ('identity_hate', LABEL)]


alldata = TabularDataset(
    path='data/train.csv',
    format='csv',
    skip_header=True,
    fields=datafields)

In [7]:
random.seed(17)
train,dev = alldata.split(split_ratio=0.9, random_state=random.getstate())

In [8]:
TEXT.build_vocab(train, vectors='glove.6B.50d', max_size=20000, min_freq=5)
LABEL.build_vocab(train)

100%|█████████▉| 399330/400000 [00:09<00:00, 40328.23it/s]


In [9]:
# generate batch iterators
random.seed(1234)
train_iterator, valid_iterator = data.BucketIterator.splits((train, dev),
                                                            batch_size=BATCH_SIZE,
                                                            device=device,
                                                            shuffle=True,
                                                            sort_key=lambda x: len(x.comment_text))

In [65]:
class NNet(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx, embeddings, text_length, lstm_hidden_size):
        super().__init__()
        
        self.embeddings = nn.Embedding.from_pretrained(embeddings, freeze=False, padding_idx=pad_idx)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_size, num_layers=1, batch_first=True)
        self.max_pool = nn.MaxPool2d((text_length,1))
        self.fc1 = nn.Linear(lstm_hidden_size, 50)
        self.fc2 = nn.Linear(50, output_dim)

    def forward(self, text):
        a1 = self.embeddings(text)
        a2 = self.lstm(a1)[0]
        a3 = self.max_pool(a2).squeeze(1)
        a4 = F.relu(self.fc1(a3))
        a5 = self.fc2(a4)
        return a5

In [66]:
OUTPUT_DIM = 6
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model = NNet(len(TEXT.vocab), EMBEDDING_SIZE, OUTPUT_DIM, PAD_IDX, TEXT.vocab.vectors,TEXT_LENGTH, 150).to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,129,156 trainable parameters


In [68]:
def fit_epoch(iterator, model, optimizer, criterion):
    train_loss = 0
    train_acc = 0
    model.train()
    all_y = []
    all_y_hat = []
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        y = torch.stack([batch.toxic,
                         batch.severe_toxic,
                         batch.obscene,
                         batch.threat,
                         batch.insult,
                         batch.identity_hate],dim=1).float().to(device)
        y_hat = model(batch.comment_text.to(device))
        loss = criterion(y_hat, y)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        all_y.append(y)
        all_y_hat.append(y_hat)
    y = torch.cat(all_y,dim=0)
    y_hat = torch.cat(all_y_hat,dim=0)
    roc = roc_auc_score(y,y_hat.sigmoid().detach())
    return train_loss / len(iterator.dataset), roc

def test_epoch(iterator, model, criterion):
    train_loss = 0
    train_acc = 0
    model.eval()
    all_y = []
    all_y_hat = []
    for i, batch in enumerate(iterator):
        y = torch.stack([batch.toxic,
                         batch.severe_toxic,
                         batch.obscene,
                         batch.threat,
                         batch.insult,
                         batch.identity_hate],dim=1).float().to(device)
        with torch.no_grad():
            y_hat = model(batch.comment_text.to(device))
        loss = criterion(y_hat, y)
        train_loss += loss.item()
        all_y.append(y)
        all_y_hat.append(y_hat)
    y = torch.cat(all_y,dim=0)
    y_hat = torch.cat(all_y_hat,dim=0)
    roc = roc_auc_score(y,y_hat.sigmoid().detach())
    return train_loss / len(iterator.dataset), roc

In [69]:
def train_n_epochs(n, lr, wd):

    criterion = nn.BCEWithLogitsLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for epoch in range(n):
        start_time = time.time()
        train_loss, train_roc = fit_epoch(train_iterator, model, optimizer, criterion)
        valid_loss, valid_roc = test_epoch(valid_iterator, model, criterion)

        secs = int(time.time() - start_time)
        mins = secs / 60
        secs = secs % 60

        print('Epoch: %d' % (epoch + 1), " | time in %d minutes, %d seconds" % (mins, secs))
        print(f'\tLoss: {train_loss:.4f}(train)\t|\troc: {train_roc :.6f} (train)')
        print(f'\tLoss: {valid_loss:.4f}(valid)\t|\troc: {valid_roc:.6f} (valid)')

In [ ]:
train_n_epochs(5,0.01,0)

Epoch: 1  | time in 5 minutes, 19 seconds
	Loss: 0.0009(train)	|	roc: 0.957808 (train)
	Loss: 0.0009(valid)	|	roc: 0.970781 (valid)
Epoch: 2  | time in 5 minutes, 15 seconds
	Loss: 0.0007(train)	|	roc: 0.978593 (train)
	Loss: 0.0008(valid)	|	roc: 0.972617 (valid)
Epoch: 3  | time in 5 minutes, 20 seconds
	Loss: 0.0007(train)	|	roc: 0.983613 (train)
	Loss: 0.0008(valid)	|	roc: 0.972922 (valid)
Epoch: 4  | time in 5 minutes, 46 seconds
	Loss: 0.0006(train)	|	roc: 0.985390 (train)
	Loss: 0.0009(valid)	|	roc: 0.970077 (valid)
